In [ ]:
import logging
import pandas as pd
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def get_sectors(conn, cur, id_state):
    query = f"SELECT cod_ibge FROM public.setores_censitarios WHERE CAST(cod_ibge AS TEXT) LIKE '{id_state}%';"
    
    try:
        cur.execute(query)
        result = [row[0] for row in cur.fetchall()]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_elderly(conn, cur, table, id_sector):
    query = f" \
        SELECT SUM(\"65_69\" + \"70_74\" + \"75_79\" + \"80_84\" + \"85_89\" + \"90_94\" + \"95_99\" + \"100\") \
        FROM public.{table} \
        WHERE id_setor_censitario = {id_sector};"
    
    try:
        cur.execute(query)
        result = cur.fetchone()[0]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_children(conn, cur, table, id_sector):
    query = f" \
        SELECT SUM(\"00_04\" + \"05_09\" + \"10_14\") \
        FROM public.{table} \
        WHERE id_setor_censitario = {id_sector};"
    
    try:
        cur.execute(query)
        result = cur.fetchone()[0]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def insert_data(conn, cur, id_sector, data):
    query = f"INSERT INTO public.indice_envelhecimento (id_setor_censitario, masculino, feminino) VALUES ({id_sector}, {data[0]}, {data[1]});"
    
    try:
        cur.execute(query)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        logger.error(f"Failed: {e}")
    else:
        conn.commit()

---

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
conn, cur = open_connection()

In [ ]:
sectors = get_sectors(conn, cur, 12) # AC

In [ ]:
logger.info(f"{len(sectors)} found.")

In [ ]:
%%time
for sector in sectors:
    male_elderly = get_elderly(conn, cur, 'populacao_residente_mas', sector)
    male_children = get_children(conn, cur, 'populacao_residente_mas', sector)
    
    female_elderly = get_elderly(conn, cur, 'populacao_residente_fem', sector)
    female_children = get_children(conn, cur, 'populacao_residente_fem', sector)
    
    try:
        male_idx = male_elderly / male_children
    except Exception as e:
        male_idx = 'NULL'
        
    try:
        female_idx = female_elderly / female_children
    except Exception as e:
        female_idx = 'NULL'
    
    # insert_data(conn, cur, sector, [male_idx, female_idx])    

In [ ]:
close_connection(conn, cur)